some observations    
===============

**vocabulary size**    
tweet tokenizer / no preprocessing = 313803

In [65]:
import pickle
import pandas as pd
import math
from collections import Counter
import sys
import csv
import string
import re
import emoji
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from itertools import chain

In [66]:
df = pd.read_csv('tweets.csv', sep='\t', usecols=[1,4], names=['id', 'tweet'])

In [59]:
#tweets = df['tweet'].tolist()

In [60]:
tokenizer = TweetTokenizer()
unicodes2remove = [
    # all kinds of quotes
    u'\u2018', u'\u2019', u'\u201a', u'\u201b', u'\u201c', \
    u'\u201d', u'\u201e', u'\u201f', u'\u2014',
    # all kinds of hyphens
    u'\u002d', u'\u058a', u'\u05be', u'\u1400', u'\u1806', \
    u'\u2010', u'\u2011', u'\u2012', u'\u2013',
    u'\u2014', u'\u2015', u'\u2e17', u'\u2e1a', u'\u2e3a', \
    u'\u2e3b', u'\u2e40', u'\u301c', u'\u3030',
    u'\u30a0', u'\ufe31', u'\ufe32', u'\ufe58', u'\ufe63', \
    u'\uff0d', u'\u00b4'
]

punctuation = string.punctuation.replace('@', '') + ''.join(unicodes2remove)
# regex to match urls (taken from the web)
urlregex = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]'
                           '|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
# keep @ to be able to recognize usernames
punctuation = string.punctuation.replace('@', '') + ''.join(unicodes2remove)
punctuation = punctuation.replace('#', '')
# a bunch of emoji unicodes
emojis = ''.join(emoji.UNICODE_EMOJI)
emojis = emojis.replace('#', '')
# combined english and german stop words
stop_words = set(stopwords.words('english') + stopwords.words('german'))

In [107]:
def clean(s):
    """
    Normalizes a string (tweet) by removing the urls, punctuation, digits,
    emojis, by putting everything to lowercase and removing the
    stop words. Tokenization is performed aswell.

    :param s the string (tweet) to clean
    :return: returns a list of cleaned tokens
    """
    s = ' '.join(s.replace('[NEWLINE]', '').split())
    s = ' '.join(s.replace('…', '...').split())
    s = urlregex.sub('', s).strip()
    s = s.translate(str.maketrans('', '', punctuation + string.digits \
                                  + emojis)).strip()
    s = ' '.join(s.split())
    s = s.lower()
    s = tokenizer.tokenize(s)
    s = [w for w in s if w not in stop_words]
    return s if s else None

In [62]:
#@ Brandon: this takes a while use pickled version instead (below)
#V = set()
#for tweet in tweets:
#    [V.add(token) for token in tokenizer.tokenize(tweet)]

#TODO: we can basically reuse our old code here which cleans and tokenized the tweets

In [63]:
tokenized = df['tweet'].apply(lambda x: tokenizer.tokenize(x))

In [83]:
V = set(chain.from_iterable(tokenized))

In [106]:
start = datetime.now()
cleaned = [clean(w) for w in w]
time = datetime.now() - start
print(cleaned)[:10]

KeyboardInterrupt: 

In [42]:
len(V)

313803

In [44]:
#dump
with open('vocabulary_naive.pickle', 'wb') as f:
    pickle.dump(V, f)

In [7]:
#load
"""
f = open('vocabulary_naive.pickle', 'rb')
V = pickle.load(f)
f.close()

In [12]:
list(V)[:10]

['armselig',
 'Feinmotorik',
 '#Abitur2018',
 'realitätsverweigerung',
 'factor-like',
 'papu',
 '275',
 'Sepsis',
 'Maffiawaschmaschine',
 'https://t.co/9JdHg2J519']

In [122]:
def tfidf(doc1, doc2, V, tweets):
    """
    d1,d2 -> list of tokens
    """
    #i think we can use this intersection here because words that are in the in one 
    #vector but not in the other result in zero multiplication
    intersect = set(doc1).intersection(set(doc2))
    c1 = Counter(doc1)#token count
    c2 = Counter(doc2)#token count
    d1 = {}
    d2 = {}
    for t in doc1:
        if t not in d1.keys() and t in intersect:
            #tf idf
            tf = c1[t]
            df = len([tweet for tweet in tweets if t in tweet]) #naive string matching
            idf = len(V) / df
            tfidf = (1 + math.log10(tf)) * (math.log10(idf))
            d1[t] = tfidf
    for t in doc2:
        if t not in d2.keys() and t in intersect:
            #tf idf
            tf = c2[t]
            
            #naive string matching
            #TODO: after tokenization and preprocessing was improved 
            #we should look if the token is in a tweet by comparing single tokens and not 
            #the whole string
            df = len([tweet for tweet in tweets if t in tweet])
            idf = len(V) / df
            tfidf = (1 + math.log10(tf)) * (math.log10(idf))
            d2[t] = tfidf
            df_tfidf = pd.DataFrame().from_dict(d1, orient='index')
            df_tfidf[1] = pd.DataFrame().from_dict(d2, orient='index')
    return df_tfidf

In [146]:
#@Brandon: its fun to play around with the those documents
doc1 = "this is a random tweet Hausarzt Affe Affe Affe".split()
doc2 = "this is random a a a a a tweet Hausarzt Hausarzt Hausarzt Hausarzt Hausarzt I think bla foo Affe".split()

In [13]:
df_tfidf = tfidf(doc1, doc2, V, tweets)
df_tfidf

NameError: name 'tfidf' is not defined

In [14]:
df.head()

,id,tweet
0,965734992633565184,@knakatani @ChikonJugular @joofford @SteveBlog...
1,965706998946893824,"@FischerKurt Lady, what´s a tumor? #KippCharts"
2,965695626150326273,@Kings_of_Metal Ohne Diagnoseverdacht ist es n...
3,965692298343444481,@GermanLetsPlay @Quentin34013799 @_Lopoopl_ @L...
4,965672579133566980,Interesting. ⬆️ pCR rate at major centers. Aut...


In [139]:
def cosine(vec1, vec2):
    nominator = 0
    denominator = 0
    vec1_length = 0
    vec2_length = 0
    for v1,v2 in zip(vec1,vec2):
        nominator += v1*v2
        vec1_length += v1*v1
        vec2_length += v2*v2
    vec1_length = math.sqrt(vec1_length)
    vec2_length = math.sqrt(vec2_length)
    denominator = vec1_length * vec2_length 
    return nominator / denominator

In [148]:
cosine(df_tfidf[0], df_tfidf[1])

0.9298679480866374